In [1]:
import numpy as np
import pandas as pd
import csv
from scamp import *
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Dense,Flatten,Softmax,Input
import keras.backend as K 

Using TensorFlow backend.


In [28]:
def creat_model():
    
    rnn_units = 12
    n_notes = 30

    input_A = keras.layers.Input(shape=(20,31), name="input_A") # (15, 20, 31)
    input_B = keras.layers.Input(shape=(1,31), name="input_B") # (15, 1, 31)

    hidden1 = keras.layers.LSTM(rnn_units, return_sequences=True)(input_A) # (15,20,12)
    hidden2 = keras.layers.LSTM(rnn_units, return_sequences=True)(hidden1) #(15,20,12)
    
    output_RNN = keras.layers.Dense(n_notes, activation='softmax', name = 'output_RNN')(hidden2) #(15,20,30)

    e = keras.layers.Dense(1, activation='tanh')(hidden2) #(15,20,1)
    e = keras.layers.Reshape([-1])(e) #(15,20)

    alpha = keras.layers.Activation('softmax')(e) #(15,20)
    c = keras.layers.Permute([2, 1])(keras.layers.RepeatVector(rnn_units)(alpha)) #(15,20,12)
    c = keras.layers.Multiply()([hidden2, c]) #(15,20,12)
    c = keras.layers.Lambda(lambda xin: K.sum(xin, axis=1), output_shape=(rnn_units,))(c) #(15,12)

    output_A = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_A')(c) #(15,30) (0, cantus)

    aux = keras.layers.SimpleRNN(rnn_units)(input_B, initial_state=c) #(15,12)
    output_B = keras.layers.Dense(n_notes, activation = 'softmax', name = 'output_B')(aux) #(15,30) (1, counter)

    model = keras.models.Model([input_A, input_B], [output_RNN, output_A, output_B])
    aux_model = keras.models.Model([input_A, input_B], [output_A, output_B])
    sub_model = keras.models.Model(input_A, output_A)
    att_model = keras.models.Model([input_A, input_B], alpha)
    
    model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy','categorical_crossentropy'],
              loss_weights=[0.2, 0.4, 0.4],optimizer="sgd")
    aux_model.compile(loss=['categorical_crossentropy', 'categorical_crossentropy'],
              optimizer="sgd")
    sub_model.compile(loss='categorical_crossentropy',
              optimizer="sgd")
    return model,aux_model,sub_model,att_model

In [29]:
model_0_1_,aux_model_0_1_, sub_model_0_1_,att_model_0_1_ = creat_model()
model_1_0_,aux_model_1_0_, sub_model_1_0_,att_model_1_0_ = creat_model()

In [30]:
model_0_1_.summary()

Model: "functional_17"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_A (InputLayer)            [(None, 20, 31)]     0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 20, 12)       2112        input_A[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LSTM)                   (None, 20, 12)       1200        lstm_4[0][0]                     
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 20, 1)        13          lstm_5[0][0]                     
______________________________________________________________________________________

In [31]:
model_0_1_.load_weights('model_0_1_weight.h5')
model_1_0_.load_weights('model_1_0_weight.h5')

In [6]:
df = pd.read_csv('Species.csv', sep=",",index_col = 0)
n = df.shape[0]
X = tf.Variable(np.zeros((2,n*2)),dtype='int32')  # any data tensor
for i in range(0,2*n,2):
    X[0,i].assign(df['Cantus'][i/2] - 43)
    X[1,i].assign(0)
    X[0,i+1].assign(df['Counter'][i/2] - 55)
    X[1,i+1].assign(1)
X

<tf.Variable 'Variable:0' shape=(2, 20000) dtype=int32, numpy=
array([[15, 12, 23, ..., 10, 13,  9],
       [ 0,  1,  0, ...,  1,  0,  1]])>

In [7]:
depth = 30
X = tf.concat([tf.one_hot(X[0,:], depth),tf.dtypes.cast(tf.reshape(X[1,:],[20000,1]),tf.float32)],1)
X

<tf.Tensor: shape=(20000, 31), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [18]:
# observation only
X0 = tf.Variable(np.zeros((n,31)),dtype='float32')
for i in range(n):
    X0[i,:].assign(X[2*i,:])
X0

<tf.Variable 'Variable:0' shape=(10000, 31) dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>

In [19]:
train_dataset = tf.data.Dataset.from_tensor_slices(X0[:9600,:])
test_dataset = tf.data.Dataset.from_tensor_slices(X0[9600:,:])

In [20]:
n_steps = 10
window_length = n_steps
dataset = train_dataset.window(window_length, shift=1, drop_remainder=True)

In [21]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [22]:
batch_size = 20
shuffle_dataset = dataset.shuffle(10000).batch(batch_size,drop_remainder=True)

In [23]:
i = 0
for window in shuffle_dataset:
    i = i + 1
    print(window.shape)
    #print(window)

print(i)

(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)
(20, 10, 31)

In [32]:
# version 1
# adapt my mind to the observation
# no action applied

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0

for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,10,31)
            initial = window

            for i in range(n): # n step forward
                counter = counter + 1

                #output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                #out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                #    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                #    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
#
                #    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                #    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
#
                #    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                #output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                #y_pred_B_0_1_  #(18,30)
                #out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                 #   pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                 #   one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                 #   out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                out_1_0_1 =  tf.reshape(output_A_1_0_,[batch_size,1,30]) #(18,1,30)

                one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
                out_1_0_1 =  np.concatenate((out_1_0_1,one),2) #(18,1,31)
                    
                #out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                 #   pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                 #   one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])

                 #   out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_1_0_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)


                #target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                #target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                #target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                History_0_1_ = sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                
                if counter%2000 == 0:
                    print('model_0_1 - loss: ', loss_0_1_/2000)
                    loss_0_1_ = 0
                    
                initial = tf.concat((initial[:,:-1,:],out_1_0_0),1)

model_0_1 - loss:  1.2399794287383556
model_0_1 - loss:  0.908804241642356
model_0_1 - loss:  0.7335450089424849
model_0_1 - loss:  0.6347356769964099
model_0_1 - loss:  0.5332564210146665
model_0_1 - loss:  0.4699535935819149
model_0_1 - loss:  0.4303412746265531
model_0_1 - loss:  0.3846493941731751
model_0_1 - loss:  0.35753752798214555
model_0_1 - loss:  0.33314108207076787
model_0_1 - loss:  0.3209772971253842
model_0_1 - loss:  0.30028758631832897
model_0_1 - loss:  0.2803442635498941
model_0_1 - loss:  0.2798271372690797
model_0_1 - loss:  0.2668539771009237
model_0_1 - loss:  0.2571242012307048
model_0_1 - loss:  0.25172699352540073
model_0_1 - loss:  0.2361077525569126
model_0_1 - loss:  0.23251615720894186
model_0_1 - loss:  0.2335380830867216
model_0_1 - loss:  0.2131701999446377
model_0_1 - loss:  0.21509645228274166
model_0_1 - loss:  0.21176635797787458
model_0_1 - loss:  0.20634426823165267
model_0_1 - loss:  0.19984655385278166
model_0_1 - loss:  0.20739763625897467
mod

In [ ]:
# version 2
# adapt my mind to the observation
# apply my assumed action to the environment
# sequential

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0

for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,10,31)
            initial = window
            
            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                
                out_0_1_0 =  tf.reshape(output_A_0_1_,[batch_size,1,30]) #(18,1,30)
                one = tf.reshape(tf.ones(batch_size),[batch_size,1,1])
                out_0_1_0 =  np.concatenate((out_0_1_0,one),2) #(18,1,31)
                
                
                
                #out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                #    # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                #    pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
#
                #    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                #    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
#
                #    out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                #output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                #out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                #    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                #    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]
#
                #    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                #    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
#
                #    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)
#
                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_0_1_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                #target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                #target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                #target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                
                History_0_1_ = sub_model_0_1_.fit(initial,target_1_0_0,verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                
                if counter%2000 == 0:
                    print('model_0_1 - loss: ', loss_0_1_/2000)
                    loss_0_1_ = 0
                   
                initial = tf.concat((initial[:,:-1,:],out_1_0_0),1)


In [ ]:
# version 3
# actively choose my action for desired results
# experiment on the environment without cost
# maximum likelihood collapse (could be modified to KL divergence)

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0

for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,10,31)
            initial = window
            
            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30) # my prediction about observation
                
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                        pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)
                        
                target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                
                target_0_1_1 = np.zeros((batch_size,30))
                for i in range(batch_size):
                    prob = np.zeros((30,))
                    for j in range(30):
                        one_hot = tf.reshape(tf.one_hot(j, 30),[1,1,30])
                        out_0_1_1 = np.concatenate((one_hot,[[[1]]]),2) #(1,1,31) # my action
                        
                        output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_0_1_1)) # my action on environment
                        y_pred_B_1_0_  #(1,30) # observation
                        prob[j] = y_pred_B_1_0_[0,pred_A_class_max_0_1_] # probability on desired observation
                        
                    the_j = np.where(prob == np.max(prob))[0][0]
                    one_hot = tf.reshape(tf.one_hot(the_j, 30),[1,30])
                    target_0_1_1[i,:] = one_hot #(18,30) # desired action

                
                History_0_1_ = aux_model_0_1_.fit([initial,out_0_1_0],[target_0_1_0,target_0_1_1],verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                
                if counter%2000 == 0:
                    print('model_0_1 - loss: ', loss_0_1_/2000)
                    loss_0_1_ = 0
                   
                initial = tf.concat((initial[:,:-1,:],out_0_1_0),1)


In [ ]:
# version 4
# actively choose my action for desired results
# cannot experiment on the environment; choose action by probability guessing

step = 15
training_times = 3;
counter = 0;
loss_0_1_ = 0

for n in range(1,step+1):
    for times in range(training_times):
        for window in shuffle_dataset: #(18,10,31)
            initial = window
            
            for i in range(n): # n step forward
                counter = counter + 1

                output_A_0_1_ = sub_model_0_1_.predict(initial) #(18,30)
                
                out_0_1_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                    for i in range(batch_size):
                        # pred_A_class_random_0_1_ = np.random.choice(30, 1, p=output_A_0_1_[i,:])[0]
                        pred_A_class_max_0_1_ = np.where(output_A_0_1_[i,:] == np.max(output_A_0_1_[i,:]))[0][0]
                        
                        # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_0_1_, 30),[1,1,30])
                        one_hot = tf.reshape(tf.one_hot(pred_A_class_max_0_1_, 30),[1,1,30])
                        
                        out_0_1_0[i:i+1,:,:] = np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)
                        
                
                output_RNN, y_pred_A_0_1_, y_pred_B_0_1_ = model_0_1_.predict((initial,out_0_1_0))
                y_pred_B_0_1_  #(18,30)
                out_0_1_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_0_1_ = np.random.choice(30, 1, p=y_pred_B_0_1_[i,:])[0]
                    pred_B_class_max_0_1_ = np.where(y_pred_B_0_1_[i,:] == np.max(y_pred_B_0_1_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_0_1_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_0_1_, 30),[1,1,30])

                    out_0_1_1[i:i+1,:,:]= np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)

                #output_A_1_0_ = sub_model_1_0_.predict(initial) #(18,30)
                #out_1_0_1 = np.zeros([batch_size,1,31]) #(18,1,31)
                #for i in range(batch_size):
                #    # pred_A_class_random_1_0_ = np.random.choice(30, 1, p=output_A_1_0_[i,:])[0]
                #    pred_A_class_max_1_0_ = np.where(output_A_1_0_[i,:] == np.max(output_A_1_0_[i,:]))[0][0]
#
                #    # one_hot = tf.reshape(tf.one_hot(pred_A_class_random_1_0_, 30),[1,1,30])
                #    one_hot = tf.reshape(tf.one_hot(pred_A_class_max_1_0_, 30),[1,1,30])
#
                #    out_1_0_1[i:i+1,:,:] = np.concatenate((one_hot,[[[1]]]),2) #(18,1,31)
#
                output_RNN, y_pred_A_1_0_, y_pred_B_1_0_ = model_1_0_.predict((initial,out_0_1_1))
                y_pred_B_1_0_  #(18,30)
                out_1_0_0 = np.zeros([batch_size,1,31]) #(18,1,31)
                for i in range(batch_size):
                    # pred_B_class_random_1_0_ = np.random.choice(30, 1, p=y_pred_B_1_0_[i,:])[0]
                    pred_B_class_max_1_0_ = np.where(y_pred_B_1_0_[i,:] == np.max(y_pred_B_1_0_[i,:]))[0][0]

                    # one_hot = tf.reshape(tf.one_hot(pred_B_class_random_1_0_, 30),[1,1,30])
                    one_hot = tf.reshape(tf.one_hot(pred_B_class_max_1_0_, 30),[1,1,30])

                    out_1_0_0[i:i+1,:,:]= np.concatenate((one_hot,[[[0]]]),2) #(18,1,31)

                #target_0_1_0 = np.reshape(out_0_1_0,[batch_size,31])[:,:-1]
                #target_0_1_1 = np.reshape(out_0_1_1,[batch_size,31])[:,:-1]
                #target_1_0_1 = np.reshape(out_1_0_1,[batch_size,31])[:,:-1]
                target_1_0_0 = np.reshape(out_1_0_0,[batch_size,31])[:,:-1]

                
                History_0_1_ = aux_model_0_1_.fit([initial,out_0_1_0],[target_0_1_0,target_0_1_1],verbose = 0)
                loss_0_1_ = loss_0_1_ + History_0_1_.history.get('loss')[0]
                
                if counter%2000 == 0:
                    print('model_0_1 - loss: ', loss_0_1_/2000)
                    loss_0_1_ = 0
                   
                initial = tf.concat((initial[:,:-1,:],out_0_1_0),1)

In [33]:
model_0_1_.save_weights('version1_mindAdapt_withoutAction_model_0_1_.h5')